In [1]:
def my_logger(func):
    
    def wrapper():
        print("Strating Wrapper")
        func()
        print("Ending Wrapper")
    
    return wrapper

@my_logger
def say_hello():
    print("Hello!!!!")


say_hello()



Strating Wrapper
Hello!!!!
Ending Wrapper


In [7]:
import time 

def timer(func):
    
    def wrapper(*args, **kwargs):
        start= time.time()
        
        result=func(*args,**kwargs)
        
        end= time.time()
        
        print(f"Finished in {end-start:.2f} seconds")
        
        return result

    return wrapper

@timer
def computation(num:int)-> int:
    time.sleep(1)
    return num*num

print(computation(10))

        

Finished in 1.00 seconds
100


In [30]:
def repeat(times=3):
    def decorator(func):
        def wrapper(*args,**kwargs):
            
            last_error=None   
                     
            for i in range(times):
                try:
                    return func(*args,**kwargs)
                    
                except Exception as e:
                    print(f"Error:{e}...Retrying ({i+1}/{times})")
                    last_error=e
                    continue
                    
            print("All retries Failed")
            
            if last_error:
                raise last_error
            else:
                raise Exception("Max retries exceeded (or 0 retries requested)")
             
        return wrapper
    return decorator

@repeat(3)
def unstable_meth(x:int)->int:
    import random
    
    if random.random() < 0.7:
        print("Crash!!")
        raise ValueError("Erm...what the Σ ??")
    
    return x*x


unstable_meth(5)
            

Crash!!
Error:Erm...what the Σ ??...Retrying (1/3)
Crash!!
Error:Erm...what the Σ ??...Retrying (2/3)
Crash!!
Error:Erm...what the Σ ??...Retrying (3/3)
All retries Failed


ValueError: Erm...what the Σ ??

In [69]:
CACHE={}

def repeat(times=3):
    def decorator(func):
        def wrapper(*args,**kwargs):
            
            key=str(args)+str(kwargs)
            
            if key in CACHE.keys():
                print("CACHE HAS FALLEN")
                return CACHE[key]
                
            last_error=None   
                     
            for i in range(times):
                try:
                    result= func(*args,**kwargs)

                    CACHE[key]=result
                    
                    return result
                    
                except Exception as e:
                    print(f"Error:{e}...Retrying ({i+1}/{times})")
                    last_error=e
                    continue
                    
            print("All retries Failed")
            
            if last_error:
                raise last_error
            else:
                raise Exception("Max retries exceeded (or 0 retries requested)")
             
        return wrapper
    return decorator

@repeat(3)
def unstable_meth(x:int)->int:
    import random
    
    if random.random() < 0.7:
        print("Crash!!")
        raise ValueError("Erm...what the Σ ??")
    
    return x*x

print(unstable_meth(5))
print(unstable_meth(5))
print(unstable_meth(5))

25
CACHE HAS FALLEN
25
CACHE HAS FALLEN
25


In [ ]:
import random
from functools import wraps


CACHE={}


def resilient_task(times:int=3, delay:float=1.0, use_cache:bool=True):
    def decorator(func):
        
        @wraps(func)
        def wrapper(*args,**kwargs):
            
            cache_key=f"{func.__name__}:{args}:{kwargs}"
            
            last_error=None
            
            if use_cache and cache_key in CACHE.keys():
                print(f"[CACHE HAS FALLEN GET TO BUNKERS!!!] Returning saved result for {cache_key}")
                return CACHE[cache_key]
            
            for i in range(times):
                try:
                    
                    start=time.time()
                    result=func(*args,**kwargs) 
                    end=time.time()
                    
                    print(f"{func.__name__} ran for {end-start:.2f}")
                    
                    CACHE[cache_key]=result
                    return result
                
                except Exception as e:
                    print(f"Error {e}... Retrying({i+1}/{times})")
                    last_error=e
                    if i<times-1:
                        time.sleep(delay)
                    continue
            
            print("All Retries Failed")
            
            if last_error:
                raise last_error
            else:
                raise Exception("Max retries exceeded (or 0 retries requested)")
        
        return wrapper
    
    return decorator


@resilient_task(3,1.0,True)
def unstable_api(data:str)->str:

    if random.random() < 0.7:
        raise ValueError("Hello Darkness MY Friend")
    return f"Processed {data}"
            

try:
    print("\n--- First Call (Might Fail/Retry) ---")
    print(unstable_api("User A"))
    
    print("\n--- Second Call (Should be Instant/Cached) ---")
    print(unstable_api("User A"))
except Exception as e:
    print(f"Final Execution Failed: {e}")
                    


--- First Call (Might Fail/Retry) ---
Error Hello Darkness MY Friend... Retrying(1/3)
Error Hello Darkness MY Friend... Retrying(2/3)
unstable_api ran for 0.00
Processed User A

--- Second Call (Should be Instant/Cached) ---
[CACHE HAS FALLEN GET TO BUNKERS!!!] Returning saved result for unstable_api:('User A',):{}
Processed User A


In [82]:
#rate limiter

import time
from functools import wraps

HISTORY={}

def rate_limit(max_calls:int,period:int|float):
    def decorator(func):
        
        @wraps(func)
        def wrapper(*args:str,**kwargs:str):
         
            func_id=func.__name__
         
            if func_id not in HISTORY:
                HISTORY[func_id]=[]
            
            now=time.time()
            
            timestamps=HISTORY[func_id]
            
            cutoff=now-period
            
            valid_timestamps=[t for t in timestamps if t>cutoff]
            
            HISTORY[func_id]=valid_timestamps
            
            if len(valid_timestamps)>=max_calls:
                raise Exception(f"Beep Boop nice try bot gotcha with rate limit(only {max_calls} calls allowed)")
            
            HISTORY[func_id].append(now)
            return func(*args,**kwargs)
        
        return wrapper
    return decorator
                    
@rate_limit(max_calls=2,period=5)
def api_call(user):
    print(f"API request sent to {user}") 
    

try:
    api_call("Alice") # Call 1 (OK)
    api_call("Bob")   # Call 2 (OK)
    print("--- Attempting 3rd call (Should Fail) ---")
    api_call("Charlie") # Call 3 (SHOULD CRASH)
except Exception as e:
    print(f"GET BLOCKED CLANKER: {e}")

print("--- Sleeping for 5 seconds ---")
time.sleep(5)

print("--- Attempting 4th call (Should Work again) ---")
api_call("Dave")

API request sent to Alice
API request sent to Bob
--- Attempting 3rd call (Should Fail) ---
GET BLOCKED CLANKER: Beep Boop nice try bot gotcha with rate limit(only 2 calls allowed)
--- Sleeping for 5 seconds ---
--- Attempting 4th call (Should Work again) ---
API request sent to Dave
